# 10_baseline_dm.ipynb

research3 phase notebook.

In [1]:

from __future__ import annotations

import json
import math
import shutil
from datetime import datetime, timezone
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import roc_auc_score

ROOT = Path('/Users/kencharoff/workspace/projects/momentum/research3')
OUT = ROOT / 'outputs'
DOCS = OUT / 'docs'
TABLES = OUT / 'tables'
FIGS = OUT / 'figs'
LOGS = OUT / 'logs'
for d in [OUT, DOCS, TABLES, FIGS, LOGS]:
    d.mkdir(parents=True, exist_ok=True)

EVIDENCE_TIER = 'secondary_web_only'
DEVIATION_FLAGS = ['trend_scanning_primary_missing']


def backup_file(path: Path) -> None:
    if path.exists():
        ts = datetime.now().strftime('%Y%m%d_%H%M%S')
        bak = path.with_name(f"{path.name}.bak.{ts}")
        shutil.copy2(path, bak)


def write_text(path: Path, text: str) -> None:
    backup_file(path)
    path.parent.mkdir(parents=True, exist_ok=True)
    path.write_text(text, encoding='utf-8')


def write_json(path: Path, payload: dict) -> None:
    write_text(path, json.dumps(payload, ensure_ascii=False, indent=2))


def write_df(path: Path, df: pd.DataFrame, index: bool = False) -> None:
    backup_file(path)
    path.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(path, index=index)


def save_fig(path: Path) -> None:
    backup_file(path)
    path.parent.mkdir(parents=True, exist_ok=True)
    plt.savefig(path, dpi=140, bbox_inches='tight')
    plt.close()


def write_manifest(
    phase_label: str,
    manifest_filename: str,
    expected_artifacts: list[str],
    parameters: dict,
    notes: list[str],
    resume_from: str,
    started_at: str | None,
) -> dict:
    found, missing = [], []
    for p in expected_artifacts:
        pp = Path(p)
        if pp.exists() and pp.stat().st_size > 0:
            found.append(str(pp))
        else:
            missing.append(str(pp))
    status = 'completed_with_deviation' if not missing else 'failed'
    payload = {
        'phase': phase_label,
        'started_at': started_at,
        'ended_at': datetime.now(timezone.utc).isoformat(),
        'status': status,
        'evidence_tier': EVIDENCE_TIER,
        'deviation_flags': DEVIATION_FLAGS,
        'resume_from': resume_from,
        'expected_artifacts': expected_artifacts,
        'found_artifacts': found,
        'missing_artifacts': missing,
        'parameters': parameters,
        'notes': notes,
    }
    write_json(LOGS / manifest_filename, payload)
    return payload


def load_inputs() -> tuple[pd.DataFrame, pd.DataFrame]:
    raw = pd.read_csv(ROOT / 'data' / 'prices.csv', header=[0, 1, 2])
    date_col = ('Price', 'Ticker', 'Date')
    dates = pd.to_datetime(raw[date_col])

    adj = raw[[c for c in raw.columns if c[0] == 'Adj Close']].copy()
    adj.columns = [c[1] for c in adj.columns]
    adj.index = dates
    adj = adj.sort_index().astype(float)

    vol = raw[[c for c in raw.columns if c[0] == 'Volume']].copy()
    vol.columns = [c[1] for c in vol.columns]
    vol.index = dates
    vol = vol.sort_index().astype(float)
    return adj, vol


def month_end_and_apply_dates(index: pd.DatetimeIndex) -> tuple[pd.DatetimeIndex, pd.Series]:
    dti = pd.DatetimeIndex(index).sort_values().unique()
    month_end = pd.Series(dti, index=dti).groupby(dti.to_period('M')).max()
    month_ends = pd.DatetimeIndex(month_end.values)
    apply_map = {}
    for d in month_ends:
        loc = dti.get_loc(d)
        if isinstance(loc, slice):
            loc = loc.start
        elif isinstance(loc, np.ndarray):
            loc = int(loc[-1])
        next_day = dti[loc + 1] if loc + 1 < len(dti) else pd.NaT
        apply_map[d] = next_day
    return month_ends, pd.Series(apply_map)


def monthly_prices_from_trading(adj: pd.DataFrame) -> tuple[pd.DataFrame, pd.Series]:
    month_ends, apply_map = month_end_and_apply_dates(adj.index)
    px_m = adj.reindex(month_ends)
    return px_m, apply_map


def dm_monthly_weights(adj: pd.DataFrame, k: int = 4) -> tuple[pd.DataFrame, pd.DataFrame, pd.Series]:
    px_m, apply_map = monthly_prices_from_trading(adj)
    score = px_m.shift(2) / px_m.shift(12) - 1.0
    w = pd.DataFrame(0.0, index=score.index, columns=score.columns)
    for dt, row in score.iterrows():
        valid = row.dropna()
        if valid.empty:
            continue
        n = min(k, len(valid))
        picks = valid.nlargest(n).index
        w.loc[dt, picks] = 1.0 / n
    return w, score, apply_map


def monthly_to_daily_weights(weights_m: pd.DataFrame, trading_index: pd.DatetimeIndex, apply_map: pd.Series) -> pd.DataFrame:
    out = pd.DataFrame(0.0, index=trading_index, columns=weights_m.columns)
    valid = [d for d in weights_m.index if d in apply_map.index and pd.notna(apply_map.loc[d])]
    if not valid:
        return out
    for i, d in enumerate(valid):
        start = apply_map.loc[d]
        if i + 1 < len(valid):
            end = apply_map.loc[valid[i + 1]]
            mask = (trading_index >= start) & (trading_index < end)
        else:
            mask = trading_index >= start
        out.loc[mask] = weights_m.loc[d].values
    return out


def backtest_daily(ret_d: pd.DataFrame, w_d: pd.DataFrame, cost_bps: float = 10.0) -> tuple[pd.Series, pd.Series, pd.Series]:
    w = w_d.reindex(index=ret_d.index, columns=ret_d.columns).fillna(0.0)
    gross = (w * ret_d).sum(axis=1).fillna(0.0)
    turnover = (0.5 * w.diff().abs().sum(axis=1)).fillna(0.0)
    net = gross - turnover * cost_bps * 1e-4
    net.name = 'net_return'
    gross.name = 'gross_return'
    turnover.name = 'turnover'
    return net, gross, turnover


def equity_curve(ret: pd.Series) -> pd.Series:
    return (1.0 + ret.fillna(0.0)).cumprod()


def drawdown_curve(ret: pd.Series) -> pd.Series:
    eq = equity_curve(ret)
    return eq / eq.cummax() - 1.0


def calc_metrics(ret: pd.Series, turnover: pd.Series | None = None) -> pd.Series:
    r = ret.dropna()
    if r.empty:
        return pd.Series({'CAGR': np.nan, 'Vol': np.nan, 'Sharpe': np.nan, 'MaxDD': np.nan, 'AvgDailyRet': np.nan, 'AvgTurnover': np.nan})
    ann = 252.0
    eq = equity_curve(r)
    years = len(r) / ann
    cagr = eq.iloc[-1] ** (1.0 / years) - 1.0 if years > 0 else np.nan
    vol = r.std(ddof=0) * np.sqrt(ann)
    sharpe = (r.mean() / r.std(ddof=0)) * np.sqrt(ann) if r.std(ddof=0) > 0 else np.nan
    maxdd = drawdown_curve(r).min()
    avg_to = turnover.mean() if turnover is not None else np.nan
    return pd.Series({'CAGR': cagr, 'Vol': vol, 'Sharpe': sharpe, 'MaxDD': maxdd, 'AvgDailyRet': r.mean(), 'AvgTurnover': avg_to})


def monthly_return_from_daily(ret_d: pd.Series) -> pd.Series:
    grp = ret_d.groupby(ret_d.index.to_period('M'))
    m = grp.apply(lambda x: (1.0 + x).prod() - 1.0)
    m.index = m.index.to_timestamp('M')
    return m


def _ols_t_r2(y: np.ndarray) -> tuple[float, float, float]:
    n = len(y)
    if n < 3 or np.isnan(y).any():
        return np.nan, np.nan, np.nan
    x = np.arange(n, dtype=float)
    x_c = x - x.mean()
    y_c = y - y.mean()
    ssx = float(np.dot(x_c, x_c))
    if ssx <= 0:
        return np.nan, np.nan, np.nan
    slope = float(np.dot(x_c, y_c) / ssx)
    yhat = y.mean() + slope * x_c
    resid = y - yhat
    sse = float(np.dot(resid, resid))
    sst = float(np.dot(y_c, y_c))
    r2 = 1.0 - sse / sst if sst > 0 else np.nan
    dof = n - 2
    if dof <= 0:
        return np.nan, r2, slope
    sigma2 = sse / dof
    se = np.sqrt(sigma2 / ssx) if sigma2 >= 0 else np.nan
    tval = slope / se if (se is not None and se > 0 and not np.isnan(se)) else np.nan
    return tval, r2, slope


def compute_trendiness_monthly(
    adj: pd.DataFrame,
    month_ends: pd.DatetimeIndex,
    lookback_min: int = 20,
    lookback_max: int = 252,
    step: int = 5,
) -> pd.DataFrame:
    rows = []
    horizons = list(range(lookback_min, lookback_max + 1, step))
    for tkr in adj.columns:
        s = np.log(adj[tkr].dropna().clip(lower=1e-12))
        for dt in month_ends:
            hist = s.loc[:dt]
            if len(hist) < lookback_min:
                continue
            best_abs = -np.inf
            best_t, best_r2, best_h = np.nan, np.nan, np.nan
            vals = hist.values
            for h in horizons:
                if len(vals) < h:
                    continue
                y = vals[-h:]
                tval, r2, _ = _ols_t_r2(y)
                if np.isnan(tval):
                    continue
                if abs(tval) > best_abs:
                    best_abs = abs(tval)
                    best_t = tval
                    best_r2 = r2
                    best_h = h
            rows.append(
                {
                    'date': dt,
                    'ticker': tkr,
                    'trend_t': best_t,
                    'trend_abs_t': abs(best_t) if not np.isnan(best_t) else np.nan,
                    'trend_r2': best_r2,
                    'trend_horizon': best_h,
                }
            )
    out = pd.DataFrame(rows)
    if out.empty:
        return out
    return out.sort_values(['date', 'ticker']).reset_index(drop=True)


def compute_term_structure(px_m: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    m6 = px_m / px_m.shift(6) - 1.0
    m12 = px_m / px_m.shift(12) - 1.0
    m18 = px_m / px_m.shift(18) - 1.0
    s = m6 - m18
    c = m6 - 2.0 * m12 + m18
    return m6, m12, m18, s, c


def _linear_residual(y: pd.Series, X: pd.DataFrame) -> tuple[pd.Series, int, float]:
    mask = y.notna() & X.notna().all(axis=1)
    n = int(mask.sum())
    if n < 3:
        return pd.Series(dtype=float), n, 1.0
    Xv = X.loc[mask].values
    yv = y.loc[mask].values
    XX = np.column_stack([np.ones(len(Xv)), Xv])
    beta = np.linalg.lstsq(XX, yv, rcond=None)[0]
    resid = yv - XX @ beta
    miss_ratio = 1.0 - n / len(y)
    return pd.Series(resid, index=y.loc[mask].index), n, float(miss_ratio)


def compute_comomentum_monthly(
    ret_d: pd.DataFrame,
    month_ends: pd.DatetimeIndex,
    factor_cols: tuple[str, str, str] = ('SPY', 'AGG', 'DBC'),
    window: int = 300,
    min_overlap: int = 252,
    max_missing_ratio: float = 0.1,
) -> pd.DataFrame:
    rows = []
    factors_all = ret_d.loc[:, list(factor_cols)]
    for dt in month_ends:
        win = ret_d.loc[:dt].tail(window)
        if len(win) < min_overlap:
            rows.append({'date': dt, 'comom_level': np.nan, 'valid_tickers': 0, 'is_feasible': False})
            continue
        fac = factors_all.reindex(win.index)
        residuals = {}
        for tkr in win.columns:
            y = win[tkr]
            resid, overlap, miss_ratio = _linear_residual(y, fac)
            if overlap >= min_overlap and miss_ratio < max_missing_ratio:
                residuals[tkr] = resid
        if len(residuals) < 4:
            rows.append({'date': dt, 'comom_level': np.nan, 'valid_tickers': len(residuals), 'is_feasible': False})
            continue
        res_df = pd.DataFrame(residuals)
        corr = res_df.corr()
        vals = corr.values[np.triu_indices_from(corr.values, k=1)]
        vals = vals[~np.isnan(vals)]
        level = float(np.mean(vals)) if len(vals) else np.nan
        rows.append({'date': dt, 'comom_level': level, 'valid_tickers': len(residuals), 'is_feasible': True})
    out = pd.DataFrame(rows).set_index('date')
    return out


def compute_stmom_turnover_monthly(adj: pd.DataFrame, vol: pd.DataFrame, month_ends: pd.DatetimeIndex) -> tuple[pd.DataFrame, pd.DataFrame]:
    px_m = adj.reindex(month_ends)
    stmom = px_m.pct_change(1)
    v_short = vol.rolling(21, min_periods=10).mean()
    v_long = vol.rolling(252, min_periods=60).mean()
    turnover = v_short / v_long - 1.0
    turnover_m = turnover.reindex(month_ends)
    return stmom, turnover_m


def compute_beta_proxy_monthly(ret_d: pd.DataFrame, month_ends: pd.DatetimeIndex, window: int = 126, min_obs: int = 100) -> tuple[pd.DataFrame, pd.DataFrame]:
    detail_rows = []
    for dt in month_ends:
        win = ret_d.loc[:dt].tail(window)
        x = win['SPY']
        for tkr in ret_d.columns:
            y = win[tkr]
            mask = x.notna() & y.notna()
            if int(mask.sum()) < min_obs:
                detail_rows.append({'date': dt, 'ticker': tkr, 'beta': np.nan, 'r2': np.nan})
                continue
            xv = x.loc[mask].values.reshape(-1, 1)
            yv = y.loc[mask].values
            model = LinearRegression().fit(xv, yv)
            beta = float(model.coef_[0])
            r2 = float(model.score(xv, yv))
            detail_rows.append({'date': dt, 'ticker': tkr, 'beta': beta, 'r2': r2})
    detail = pd.DataFrame(detail_rows)
    agg = detail.groupby('date').agg(beta_mean=('beta', 'mean'), r2_mean=('r2', 'mean'))
    agg['delta_beta_mean'] = agg['beta_mean'].diff()
    agg['fit_deterioration_mean'] = (agg['r2_mean'].rolling(3, min_periods=2).mean() - agg['r2_mean']).clip(lower=0)
    return detail, agg


def robust_scale(series: pd.Series, higher_is_better: bool, floor: float = 0.7, cap: float = 1.0) -> pd.Series:
    s = series.copy().astype(float)
    q20, q80 = s.quantile(0.2), s.quantile(0.8)
    if np.isnan(q20) or np.isnan(q80) or q80 == q20:
        return pd.Series(1.0, index=s.index)
    if higher_is_better:
        x = (s - q20) / (q80 - q20)
    else:
        x = (q80 - s) / (q80 - q20)
    x = x.clip(0, 1)
    out = floor + (cap - floor) * x
    return out.fillna(1.0)


def combine_scales(*scales: pd.Series) -> pd.Series:
    df = pd.concat(scales, axis=1)
    return df.min(axis=1, skipna=True).fillna(1.0)


def stmom_entry_filter(base_weights: pd.DataFrame, stmom: pd.DataFrame, turnover: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
    filt = pd.DataFrame(0.0, index=base_weights.index, columns=base_weights.columns)
    rows = []
    for i, dt in enumerate(base_weights.index):
        cand = base_weights.loc[dt].copy()
        prev = filt.iloc[i - 1] if i > 0 else pd.Series(0.0, index=base_weights.columns)
        st = stmom.reindex(base_weights.index).loc[dt]
        to = turnover.reindex(base_weights.index).loc[dt]
        new_entries = 0
        blocked = 0
        for tkr in base_weights.columns:
            if cand[tkr] > 0 and prev[tkr] <= 0:
                new_entries += 1
                allow = bool(pd.notna(st[tkr]) and pd.notna(to[tkr]) and (st[tkr] > 0.0) and (to[tkr] > 0.0))
                if not allow:
                    cand[tkr] = 0.0
                    blocked += 1
        filt.loc[dt] = cand
        rows.append({'date': dt, 'new_entries': new_entries, 'blocked_entries': blocked, 'block_ratio': blocked / new_entries if new_entries else 0.0})
    stats = pd.DataFrame(rows).set_index('date')
    return filt, stats


def compute_signal_bundle(adj: pd.DataFrame, vol: pd.DataFrame, use_cache: bool = True) -> dict:
    panel_path = TABLES / 'intermediate_signal_panel.csv'
    trend_path = TABLES / 'intermediate_trend_ticker.csv'
    st_path = TABLES / 'intermediate_stmom_ticker.csv'
    to_path = TABLES / 'intermediate_turnover_ticker.csv'
    comom_path = TABLES / 'intermediate_comom_monthly.csv'
    beta_detail_path = TABLES / 'intermediate_beta_detail.csv'

    if use_cache and all(p.exists() for p in [panel_path, trend_path, st_path, to_path, comom_path, beta_detail_path]):
        panel = pd.read_csv(panel_path, index_col=0, parse_dates=True)
        trend_ticker = pd.read_csv(trend_path, parse_dates=['date'])
        stmom = pd.read_csv(st_path, index_col=0, parse_dates=True)
        turnover = pd.read_csv(to_path, index_col=0, parse_dates=True)
        comom = pd.read_csv(comom_path, index_col=0, parse_dates=True)
        beta_detail = pd.read_csv(beta_detail_path, parse_dates=['date'])
        return {
            'panel': panel,
            'trend_ticker': trend_ticker,
            'stmom': stmom,
            'turnover': turnover,
            'comom': comom,
            'beta_detail': beta_detail,
        }

    month_ends, _ = month_end_and_apply_dates(adj.index)
    px_m = adj.reindex(month_ends)
    ret_d = adj.pct_change().replace([np.inf, -np.inf], np.nan)

    trend_ticker = compute_trendiness_monthly(adj, month_ends)
    trend_agg = trend_ticker.groupby('date').agg(
        trend_abs_t_mean=('trend_abs_t', 'mean'),
        trend_r2_mean=('trend_r2', 'mean'),
        trend_horizon_mean=('trend_horizon', 'mean'),
    )

    m6, m12, m18, s, c = compute_term_structure(px_m)
    term_agg = pd.DataFrame(
        {
            'M6_mean': m6.mean(axis=1),
            'M12_mean': m12.mean(axis=1),
            'M18_mean': m18.mean(axis=1),
            'S_mean': s.mean(axis=1),
            'C_mean': c.mean(axis=1),
        }
    )

    comom = compute_comomentum_monthly(ret_d, month_ends)

    stmom, turnover = compute_stmom_turnover_monthly(adj, vol, month_ends)
    st_agg = pd.DataFrame({'stmom_mean': stmom.mean(axis=1), 'turnover_mean': turnover.mean(axis=1)})

    beta_detail, beta_agg = compute_beta_proxy_monthly(ret_d, month_ends)

    panel = pd.concat(
        [trend_agg, term_agg, comom[['comom_level']], st_agg, beta_agg[['beta_mean', 'delta_beta_mean', 'fit_deterioration_mean']]],
        axis=1,
    ).sort_index()

    write_df(panel_path, panel, index=True)
    write_df(trend_path, trend_ticker, index=False)
    write_df(st_path, stmom, index=True)
    write_df(to_path, turnover, index=True)
    write_df(comom_path, comom, index=True)
    write_df(beta_detail_path, beta_detail, index=False)

    return {
        'panel': panel,
        'trend_ticker': trend_ticker,
        'stmom': stmom,
        'turnover': turnover,
        'comom': comom,
        'beta_detail': beta_detail,
    }


def load_or_compute_base_dm(adj: pd.DataFrame, use_cache: bool = True) -> dict:
    w_path = TABLES / 'intermediate_base_weights_monthly.csv'
    s_path = TABLES / 'intermediate_dm_score_monthly.csv'
    apply_path = TABLES / 'intermediate_apply_map.csv'
    if use_cache and all(p.exists() for p in [w_path, s_path, apply_path]):
        w = pd.read_csv(w_path, index_col=0, parse_dates=True)
        score = pd.read_csv(s_path, index_col=0, parse_dates=True)
        apply = pd.read_csv(apply_path, index_col=0, parse_dates=True).iloc[:, 0]
        return {'weights': w, 'score': score, 'apply_map': apply}

    w, score, apply_map = dm_monthly_weights(adj, k=4)
    write_df(w_path, w, index=True)
    write_df(s_path, score, index=True)
    write_df(apply_path, apply_map.to_frame('apply_date'), index=True)
    return {'weights': w, 'score': score, 'apply_map': apply_map}


def run_strategy_from_monthly(ret_d: pd.DataFrame, w_m: pd.DataFrame, apply_map: pd.Series, cost_bps: float = 10.0) -> dict:
    w_d = monthly_to_daily_weights(w_m, ret_d.index, apply_map)
    net, gross, turnover = backtest_daily(ret_d, w_d, cost_bps=cost_bps)
    return {'weights_daily': w_d, 'net': net, 'gross': gross, 'turnover': turnover}


def plot_equity(series_map: dict[str, pd.Series], title: str, out_path: Path) -> None:
    plt.figure(figsize=(11, 5))
    for name, ser in series_map.items():
        plt.plot(equity_curve(ser), label=name)
    plt.title(title)
    plt.legend()
    plt.grid(alpha=0.3)
    save_fig(out_path)


def plot_drawdown(series_map: dict[str, pd.Series], title: str, out_path: Path) -> None:
    plt.figure(figsize=(11, 4.5))
    for name, ser in series_map.items():
        plt.plot(drawdown_curve(ser), label=name)
    plt.title(title)
    plt.legend()
    plt.grid(alpha=0.3)
    save_fig(out_path)


def not_available_figure(out_path: Path, title: str) -> None:
    plt.figure(figsize=(8, 3.5))
    ax = plt.gca()
    ax.axis('off')
    ax.text(0.5, 0.55, 'not available', ha='center', va='center', fontsize=18)
    ax.set_title(title)
    save_fig(out_path)


def scale_pack(panel: pd.DataFrame) -> dict[str, pd.Series]:
    trend = robust_scale(panel['trend_abs_t_mean'], higher_is_better=True, floor=0.7, cap=1.0)
    term_score = panel['S_mean'] - panel['C_mean'].abs()
    term = robust_scale(term_score, higher_is_better=True, floor=0.7, cap=1.0)
    comom = robust_scale(panel['comom_level'], higher_is_better=False, floor=0.7, cap=1.0)
    beta_risk = panel['delta_beta_mean'].abs().fillna(0.0) + panel['fit_deterioration_mean'].fillna(0.0)
    beta = robust_scale(beta_risk, higher_is_better=False, floor=0.7, cap=1.0)
    return {'trend': trend, 'term': term, 'comom': comom, 'beta': beta, 'beta_risk': beta_risk}


def tail_stats(ret: pd.Series) -> pd.Series:
    r = ret.dropna()
    if r.empty:
        return pd.Series({'VaR5': np.nan, 'CVaR5': np.nan, 'WorstDay': np.nan, 'WorstMonth': np.nan})
    var5 = float(np.nanpercentile(r, 5))
    cvar5 = float(r[r <= var5].mean()) if (r <= var5).any() else np.nan
    worst_day = float(r.min())
    m = monthly_return_from_daily(r)
    worst_month = float(m.min()) if len(m) else np.nan
    return pd.Series({'VaR5': var5, 'CVaR5': cvar5, 'WorstDay': worst_day, 'WorstMonth': worst_month})


In [2]:

started_at = datetime.now(timezone.utc).isoformat()
adj, vol = load_inputs()
ret_d = adj.pct_change().replace([np.inf, -np.inf], np.nan)
base = load_or_compute_base_dm(adj, use_cache=True)
w_base = base['weights']
apply_map = base['apply_map']

# DM
res_dm = run_strategy_from_monthly(ret_d, w_base, apply_map, cost_bps=10)

# EQW benchmark (daily available universe)
avail = ret_d.notna().astype(float)
eqw_daily = avail.div(avail.sum(axis=1), axis=0).fillna(0.0)
eqw_net, eqw_gross, eqw_to = backtest_daily(ret_d, eqw_daily, cost_bps=10)

metrics = pd.DataFrame(
    {
        'DM': calc_metrics(res_dm['net'], res_dm['turnover']),
        'EQW': calc_metrics(eqw_net, eqw_to),
    }
).T
metrics.index.name = 'strategy'
write_df(TABLES / 'metrics_baseline.csv', metrics, index=True)

positions_sample = w_base.tail(24).reset_index().rename(columns={'index': 'month_end'})
write_df(TABLES / 'positions_sample.csv', positions_sample, index=False)

plot_equity({'DM': res_dm['net'], 'EQW': eqw_net}, 'Baseline Equity Curve', FIGS / 'equity_curve_baseline.png')
plot_drawdown({'DM': res_dm['net'], 'EQW': eqw_net}, 'Baseline Drawdown', FIGS / 'drawdown_baseline.png')

# intermediate for later phases
write_df(TABLES / 'intermediate_dm_daily_returns.csv', pd.DataFrame({'DM': res_dm['net'], 'EQW': eqw_net}), index=True)
write_df(TABLES / 'intermediate_dm_turnover_daily.csv', pd.DataFrame({'DM': res_dm['turnover'], 'EQW': eqw_to}), index=True)

expected = [
    str(LOGS / 'run_manifest_phase1.json'),
    str(TABLES / 'metrics_baseline.csv'),
    str(TABLES / 'positions_sample.csv'),
    str(FIGS / 'equity_curve_baseline.png'),
    str(FIGS / 'drawdown_baseline.png'),
]
write_manifest(
    phase_label='1',
    manifest_filename='run_manifest_phase1.json',
    expected_artifacts=expected,
    parameters={'k': 4, 'cost_bps': 10, 'dm_filter': '12m-2m'},
    notes=['Phase 1 completed (DM baseline vs EQW).'],
    resume_from='phase0',
    started_at=started_at,
)
print('Phase 1 completed')


Phase 1 completed
